In [84]:
from scipy.stats import norm
from pandas import DataFrame
import numpy as np
import pandas as pd

def bsCall(S, K, r, s, T):
    d1 = (np.log(S/K) + (r + 0.5*s**2)*T)/(s*np.sqrt(T))
    d2 = d1 - s*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    
def bsPut(S, K, r, s, T):
    d1 = (np.log(S/K) + (r + 0.5*s**2)*T)/(s*np.sqrt(T))
    d2 = d1 - s*np.sqrt(T)
    return -S*norm.cdf(-d1) + K*np.exp(-r*T)*norm.cdf(-d2)

vol = pd.read_csv('IV.csv', parse_dates=['Date']).iloc[1:, :].set_index('Date')
spx = pd.read_csv('data.csv', parse_dates=['Date'])[['Date', 'SPX']].set_index('Date')
param = pd.read_csv('LIBOR.csv', header=5, parse_dates=[0]).iloc[:, :2].set_index('Date').rename(columns={'PX_ASK': '1ML'}).merge(spx, on='Date')

dates = param.index.intersection(vol.index)
param = param.loc[dates]
param = param.assign(FWD=np.exp(param['1ML']/1200)*param['SPX'])
vol = vol.loc[dates]
strike = DataFrame(np.array([param['FWD'] + Delta for Delta in [-90, -75, -60, -50, -40, -25, -10, 0, 10, 25, 40, 50, 60, 75, 90]]).T, 
                  columns=vol.columns, index=vol.index)

m, n = vol.shape
pv = np.empty_like(vol)
for i in range(m)[-5:]:
    for j in range(n):
        if vol.columns[j][0] == 'C':
            pv[i, j] = bsCall(param['SPX'].iloc[i], strike.iloc[i, j], param['1ML'].iloc[i]/100, vol.iloc[i, j]/100, 1/12)
        else:
            pv[i, j] = bsPut(param['SPX'].iloc[i], strike.iloc[i, j], param['1ML'].iloc[i]/100, vol.iloc[i, j]/100, 1/12)
        
DataFrame(pv[-5:], columns=vol.columns, index=vol.index[-5:])


,PUT_90,PUT_75,PUT_60,PUT_50,PUT_40,PUT_25,PUT_10,ATM,CALL_10,CALL_25,CALL_40,CALL_50,CALL_60,CALL_75,CALL_90
Date,,,,,,,,,,,,,,,
2021-02-24,48.325957,32.296234,42.015552,49.486343,60.250382,80.913456,116.916465,72.199746,55.191807,51.888455,50.551336,50.371343,53.213435,61.283371,83.589657
2021-02-25,59.755470,47.929702,64.045481,76.715573,90.233688,117.201072,158.896512,100.654375,66.923280,69.059086,73.487957,78.000543,83.081831,96.910734,124.853990
2021-02-26,57.679107,47.927257,65.580558,77.705895,92.518356,117.591548,155.612737,101.502784,67.708384,69.061458,75.079342,79.010628,85.374584,97.305249,121.614727
2021-03-01,49.488616,36.084807,47.547005,55.864101,67.938592,92.091898,130.872869,78.479550,60.114657,56.112845,56.344434,56.842535,60.834388,72.168337,97.213360
2021-03-02,50.993037,39.453408,51.633654,61.745971,73.098640,96.821217,133.539959,85.010512,61.684369,59.832431,60.610625,62.813483,65.971026,76.808682,99.840823


In [79]:
vol.shape

(3998, 15)

In [52]:
vol.columns

Index(['PUT_90', 'PUT_75', 'PUT_60', 'PUT_50', 'PUT_40', 'PUT_25', 'PUT_10',
       'ATM', 'CALL_10', 'CALL_25', 'CALL_40', 'CALL_50', 'CALL_60', 'CALL_75',
       'CALL_90'],
      dtype='object')

In [50]:
vol

,PUT_90,PUT_75,PUT_60,PUT_50,PUT_40,PUT_25,PUT_10,ATM,CALL_10,CALL_25,CALL_40,CALL_50,CALL_60,CALL_75,CALL_90
Date,,,,,,,,,,,,,,,
2005-01-04,13.4527,11.73719,11.80619,11.85213,12.50881,13.47850,14.38300,11.9063,11.66763,11.73713,11.80613,11.85188,12.50688,13.47681,14.38138
2005-01-05,13.4918,11.79988,11.94081,12.03381,12.68488,13.65469,14.56019,12.0342,11.65950,11.80213,11.94288,12.03600,12.70050,13.66919,14.57369
2005-01-06,12.9494,11.64731,11.57200,11.52163,12.08738,12.93950,13.73881,11.5200,11.72131,11.64650,11.57131,11.52081,12.09638,12.94788,13.74669
2005-01-07,12.8366,11.32900,11.22613,11.15688,11.69213,12.49850,13.25600,11.1555,11.42988,11.32788,11.22488,11.15581,11.70131,12.50713,13.26413
2005-01-10,12.6316,11.27913,11.14300,11.05131,11.77613,12.83050,13.80463,11.0877,11.41363,11.27919,11.14313,11.05150,11.77469,12.82950,13.80400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24,19.2316,14.02650,15.12210,15.97000,17.48550,20.61360,26.99510,15.9723,13.26920,14.02650,15.12210,15.97000,17.48550,20.61360,26.99510
2021-02-25,22.5520,18.29700,20.78060,22.76620,24.86680,29.42790,37.21970,22.8280,16.26400,18.29700,20.78060,22.76620,24.86680,29.42790,37.21970
2021-02-26,22.1481,18.38460,21.24550,23.10740,25.51530,29.65850,36.64730,23.1304,16.52050,18.38460,21.24550,23.10740,25.51530,29.65850,36.64730


In [82]:
df = pd.merge(rates, vol, on='Date', how='inner').set_index('Date')
df = df.assign(SPX=pd.read_csv('data.csv', parse_dates=['Date'])[['Date', 'SPX']].set_index('Date').loc[df.index])
df.head()



,LIBOR1M,PUT_90,PUT_75,PUT_60,PUT_50,PUT_40,PUT_25,PUT_10,ATM,CALL_10,CALL_25,CALL_40,CALL_50,CALL_60,CALL_75,CALL_90,SPX
Date,,,,,,,,,,,,,,,,,
2005-01-04,2.40,13.4527,11.73719,11.80619,11.85213,12.50881,13.47850,14.38300,11.9063,11.66763,11.73713,11.80613,11.85188,12.50688,13.47681,14.38138,1188.05
2005-01-05,2.40,13.4918,11.79988,11.94081,12.03381,12.68488,13.65469,14.56019,12.0342,11.65950,11.80213,11.94288,12.03600,12.70050,13.66919,14.57369,1183.74
2005-01-06,2.42,12.9494,11.64731,11.57200,11.52163,12.08738,12.93950,13.73881,11.5200,11.72131,11.64650,11.57131,11.52081,12.09638,12.94788,13.74669,1187.89
2005-01-07,2.43,12.8366,11.32900,11.22613,11.15688,11.69213,12.49850,13.25600,11.1555,11.42988,11.32788,11.22488,11.15581,11.70131,12.50713,13.26413,1186.19
2005-01-10,2.44,12.6316,11.27913,11.14300,11.05131,11.77613,12.83050,13.80463,11.0877,11.41363,11.27919,11.14313,11.05150,11.77469,12.82950,13.80400,1190.24
